In [2]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
import torch.nn as nn
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [47]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
import cv2
from os import path as osp
import numpy as np

In [48]:
"""
parser = argparse.ArgumentParser(description='VAE MNIST Example')
parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                    help='input batch size for training (default: 128)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='enables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
parser.add_argument('--hidden-size', type=int, default=20, metavar='N',
                    help='how big is z')
parser.add_argument('--intermediate-size', type=int, default=128, metavar='N',
                    help='how big is linear around z')
parser.add_argument('--widen-factor', type=int, default=1, metavar='N',
                     help='how wide is the model')
args = parser.parse_args()
"""
args = {"batch-size":128, "epochs": 10, "no-cuda": False, "seed":1, "log_interval": 10, "hidden_size":20, "intermediate_size":128, "widen-factor":1}

args["cuda"] = not args["no-cuda"] and torch.cuda.is_available()

In [49]:
torch.manual_seed(args["seed"])
if args["cuda"]:
    torch.cuda.manual_seed(args["seed"])

In [50]:
class MomentsDataset(Dataset):
    def __init__(self, data_path, mode):
        self.data_path = data_path
        
        if mode == 'train':
            self.df = pd.read_csv(self.data_path + 'trainingSet_filtered.csv', header=None, skiprows=(lambda x:x > 2047))
        elif mode == 'test':
            self.df = pd.read_csv(self.data_path + 'trainingSet_filtered.csv', header=None, skiprows=(lambda x:x < 2048 or x > 2303))
        self.class_to_index = {}
        actions = [file_name.split('/')[0] for file_name in
                list(self.df[self.df.columns[1]])]

        actions = set(actions)

        for i, action in enumerate(actions):
            self.class_to_index[action] = i


    def get_num_classes(self):
        return len(self.class_to_index)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        #print('---- df loc-----')
        #print(self.df.iloc[idx])
        #print('idx', idx)
        file_loc, label, _, _ = self.df.iloc[idx]
        #print('file loc', file_loc)

        full_path = osp.join(self.data_path, 'training', file_loc)
        dir_path = full_path.split('.')[0]

        im_data = []
        images = os.listdir(dir_path)
        for image in images:
            im = cv2.imread(osp.join(dir_path, image))
            resized_image = cv2.resize(im, (128, 128))
            #gray_im = rgb2gray(resized_image)

            im_data.append(resized_image)

        im_data = np.array(im_data)
        im_data = im_data / 255.0
        im_data = im_data[:90]
        if len(im_data) < 90:
            paste_im_data = np.zeros((90, 128, 128, 3)) #potential reshape errors
            paste_im_data[:len(im_data)] = im_data
            paste_im_data[len(im_data):] = im_data[-1]
            im_data = paste_im_data
            

        index = self.class_to_index[label]

        ret_info = {
                'images': im_data[0],
                'label': index
                }

        return ret_info

In [51]:
dataset_path = '/hdd/datasets/Moments_in_Time_Mini/'

ds_train = MomentsDataset(dataset_path, 'train')
ds_test = MomentsDataset(dataset_path, 'test')

#code_size = ds.get_num_classes()

#label_count = ds.get_num_classes()

train_loader = DataLoader(ds_train, batch_size=args["batch-size"], shuffle=True, num_workers=1, pin_memory=True)
test_loader = DataLoader(ds_test, batch_size=args["batch-size"], shuffle=False, num_workers=1, pin_memory=True)

In [52]:
#kwargs = {'num_workers': 1, 'pin_memory': True} if args["cuda"] else {}

'''
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=True, download=True,
                     transform=transforms.ToTensor()),
    batch_size=args["batch-size"], shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=False, transform=transforms.ToTensor()),
    batch_size=args["batch-size"], shuffle=False, **kwargs)
'''

'\ntrain_loader = torch.utils.data.DataLoader(\n    datasets.CIFAR10(\'../data\', train=True, download=True,\n                     transform=transforms.ToTensor()),\n    batch_size=args["batch-size"], shuffle=True, **kwargs)\ntest_loader = torch.utils.data.DataLoader(\n    datasets.CIFAR10(\'../data\', train=False, transform=transforms.ToTensor()),\n    batch_size=args["batch-size"], shuffle=False, **kwargs)\n'

In [59]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        # Encoder
        self.conv1 = nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(3, 32, kernel_size=2, stride=2, padding=0)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(131072, args["intermediate_size"])
        #16*16*32

        # Latent space
        self.fc21 = nn.Linear(args["intermediate_size"], args["hidden_size"])
        self.fc22 = nn.Linear(args["intermediate_size"], args["hidden_size"])

        # Decoder
        self.fc3 = nn.Linear(args["hidden_size"], args["intermediate_size"])
        #8192
        self.fc4 = nn.Linear(args["intermediate_size"], 131072)
        self.deconv1 = nn.ConvTranspose2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.deconv2 = nn.ConvTranspose2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.deconv3 = nn.ConvTranspose2d(32, 32, kernel_size=2, stride=2, padding=0)
        self.conv5 = nn.Conv2d(32, 3, kernel_size=3, stride=1, padding=1)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        out = self.relu(self.conv1(x))
        out = self.relu(self.conv2(out))
        out = self.relu(self.conv3(out))
        out = self.relu(self.conv4(out))
        out = out.view(out.size(0), -1)
        h1 = self.relu(self.fc1(out))
       
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h3 = self.relu(self.fc3(z))
        out = self.relu(self.fc4(h3))
        # import pdb; pdb.set_trace()
        #64,64
        out = out.view(out.size(0), 32, 64, 64)
        out = self.relu(self.deconv1(out))
        out = self.relu(self.deconv2(out))
        out = self.relu(self.deconv3(out))
        out = self.sigmoid(self.conv5(out))
        return out

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE()
if args["cuda"]:
    model.cuda()
optimizer = optim.RMSprop(model.parameters(), lr=1e-3)


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x.view(-1, 32 * 32 * 3),
                                 x.view(-1, 32 * 32 * 3), size_average=False)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


def train(epoch):
    model.train()
    train_loss = 0

    for batch_idx, data_ in enumerate(train_loader):
        data = data_["images"]
        data = data.permute(0,3,2,1)
        data = Variable(data)
        if args["cuda"]:
            data = data.cuda()
        optimizer.zero_grad()
        data = data.float()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        #train_loss += loss.data[0]
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % args["log_interval"] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    for i, data_ in enumerate(test_loader):
        data = data_["images"]
        data = data.permute(0,3,2,1)
        if args["cuda"]:
            data = data.cuda()
        data = Variable(data, volatile=True)
        data = data.float()
        recon_batch, mu, logvar = model(data)
        #test_loss += loss_function(recon_batch, data, mu, logvar).data[0]
        test_loss += loss_function(recon_batch, data, mu, logvar).item()
        if epoch == args["epochs"] and i == 0:
            n = min(data.size(0), 8)
            comparison = torch.cat([data[:n],
                                   recon_batch[:n]])
            save_image(comparison.data.cpu(),
                       '3d_results/reconstruction_' + str(epoch) +
                       '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [60]:
for epoch in range(1, args["epochs"] + 1):
    train(epoch)
    test(epoch)
    if epoch == args["epochs"]:
        sample = Variable(torch.randn(64, args.hidden_size))
        if args["cuda"]:
            sample = sample.cuda()
        sample = model.decode(sample).cpu()
        save_image(sample.data.view(64, 3, 32, 32),
                   '3d_results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/2048 (0%)]	Loss: 34033.960938
Train Epoch: 1 [1280/2048 (62%)]	Loss: 30035.183594
====> Epoch: 1 Average loss: 3840864.3478


/opt/conda/envs/pt/lib/python3.5/site-packages/ipykernel_launcher.py:121: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


====> Test set loss: 25496.3613
Train Epoch: 2 [0/2048 (0%)]	Loss: 24878.333984
Train Epoch: 2 [1280/2048 (62%)]	Loss: 23296.400391
====> Epoch: 2 Average loss: 23641.2401
====> Test set loss: 23581.3486
Train Epoch: 3 [0/2048 (0%)]	Loss: 23092.419922
Train Epoch: 3 [1280/2048 (62%)]	Loss: 22420.818359
====> Epoch: 3 Average loss: 22978.5455
====> Test set loss: 23643.7500
Train Epoch: 4 [0/2048 (0%)]	Loss: 23144.908203


KeyboardInterrupt: 